## Data preprocessing codes

In [4]:
%matplotlib inline

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm

import utils

In [5]:
#loading all dataframes 
df = utils.load('fma_metadata/tracks.csv')
genres_df = utils.load('fma_metadata/genres.csv')
echonest_df = utils.load('fma_metadata/echonest.csv')

#### DF1 df_global: raw dataset (outliers not cleaned) with all tracks without echonest data (100'000 + rows)

In [6]:
df_global = df.copy()

In [7]:
# Data cleaning
#data cleaning CODE
df_global.drop(df_global.loc[(df_global.album.comments == -1),:].index, inplace = True)
df_global.drop(df_global.loc[df_global.artist.comments==-1].index, inplace=True)

#for rows where date_released > date_created: replace values with values from date_created
df_global.loc[df_global.album.date_released > df_global.album.date_created, ('album', 'date_released')] = df_global.loc[df_global.album.date_released > df_global.album.date_created, ('album', 'date_created')]

#encode variable into a binary attribute: 1 if engineer is present, 0 if not
df_global.loc[df_global.album.engineer.isna(),('album','engineer')] = 0
df_global.loc[df_global.album.engineer.notna(),('album','engineer')] = 1

#encode variable into a binary attribute: 1 if producer is present, 0 if not
df_global.loc[df_global.album.producer.isna(),('album','producer')] = 0
df_global.loc[df_global.album.producer.notna(),('album','producer')] = 1

In [8]:
#drop entire columns
df_global.drop(columns = [('album','information')], inplace = True)
df_global.drop(columns = [('album','tags')], inplace = True)
df_global.drop(columns= [('album','date_created')], inplace =  True)

df_global.drop(columns = [('artist','active_year_begin')], inplace = True)
df_global.drop(columns = [('artist','active_year_end')], inplace = True)
df_global.drop(columns = [('artist','associated_labels')], inplace = True)
df_global.drop(columns = [('artist','bio')], inplace = True)
df_global.drop(columns = [('artist','date_created')], inplace = True)
df_global.drop(columns = [('artist','location')], inplace = True)
df_global.drop(columns = [('artist','latitude')], inplace = True)
df_global.drop(columns = [('artist','longitude')], inplace = True)
df_global.drop(columns = [('artist','name')], inplace = True)
df_global.drop(columns = [('artist','website')], inplace = True)
df_global.drop(columns = [('artist','wikipedia_page')], inplace = True)

df_global.drop(columns = [('track','composer')], inplace = True)
df_global.drop(columns = [('track','date_created')], inplace = True)
df_global.drop(columns = [('track','lyricist')], inplace = True)
df_global.drop(columns = [('track','tags')], inplace = True)
df_global.drop(columns = [('track','information')], inplace = True)
df_global.drop(columns = [('track','publisher')], inplace = True)
df_global.drop(columns = [('track','number')], inplace = True)
df_global.drop(columns = [('track','title')], inplace = True)

In [6]:
#use track.date_recorded to impute album.date_released. Then drop track.date_recorded
df_global.loc[(df_global.track.date_recorded.notna()) & (df_global.album.date_released.isna()), ('album','date_released')] = df_global.loc[(df_global.track.date_recorded.notna()) & (df_global.album.date_released.isna()), ('track','date_recorded')]
df_global.drop(columns = [('track','date_recorded')], inplace = True)

#genres
#1 -> drop rows with no genres at all
df_global.drop(df_global.loc[df_global.track.genres_all.str.len() == 0].index, inplace = True)
#2 -> clean genre top  all: abbiamo creato una nuova colonna uguale a genres_all dalla quale abbiamo rimosso tutti gli id che non sono top level and ADD NEW COLUMN CREATED TO df_global
genres_top_list = list(set(genres_df.top_level.to_list()))
genre_top_all = pd.DataFrame(df_global.track.genres_all)
for i, lista in genre_top_all.iterrows():
    temp_list = []
    for item in lista[0]:
        if item not in genres_top_list:
            temp_list.append(item)
    for j in temp_list:
        lista[0].remove(j)
df_global.track = df_global.track.join(genre_top_all.rename(columns={'genres_all' : 'genres_top_all'}),how = 'outer')
#3 -> fill genre_top empty with "Mixed" (because they have mixed genres)
df_global.loc[:,('track','genre_top')].fillna('Mixed',inplace = True)

#building global_track df_global
global_track = df_global.copy()

In [7]:
global_track.columns = global_track.columns.map('_'.join).str.strip('_')

In [8]:
#LE LISTE CON VALORI NULLI SONO LISTE VUOTE
list_null = ['artist_members','artist_related_projects']

for column in global_track.loc[:,list_null]:
    global_track.loc[global_track[column].notnull(), column] = global_track.loc[global_track[column].notnull(), column].apply(lambda x: x.split(", "))
    global_track.loc[global_track[column].isnull(), column] = global_track.loc[global_track[column].isnull(),column].apply(lambda x: [])

In [ ]:
#REPLACING MISSING VALUES BY IDS
global_track.loc[:,'track_license'] = global_track.groupby('artist_id')['track_license'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
global_track.loc[:,'album_type'] = global_track.groupby('album_id')['album_type'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
global_track.loc[:,'track_language_code'] = global_track.groupby('album_id')['track_language_code'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
global_track.loc[:,'track_language_code'] = global_track.groupby('artist_id')['track_language_code'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
global_track.loc[:,'album_date_released'] = global_track.groupby('album_id')['album_date_released'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)

In [ ]:
global_track.to_csv('global_track.csv', index = True)

#### DF2 echo_track: clean dataframe (tracks with echonest info, 10'000 + rows)

In [ ]:
df_echo = df.copy()

In [ ]:
# Data cleaning
#data cleaning CODE
df_echo.drop(df_echo.loc[(df_echo.album.comments == -1),:].index, inplace = True)
df_echo.drop(df_echo.loc[df_echo.artist.comments==-1].index, inplace=True)

#for rows where date_released > date_created: replace values with values from date_created
df_echo.loc[df_echo.album.date_released > df_echo.album.date_created, ('album', 'date_released')] = df_echo.loc[df_echo.album.date_released > df_echo.album.date_created, ('album', 'date_created')]
#drop('album','date_created')

#encode variable into a binary attribute: 1 if engineer is present, 0 if not
df_echo.loc[df_echo.album.engineer.isna(),('album','engineer')] = 0
df_echo.loc[df_echo.album.engineer.notna(),('album','engineer')] = 1

#encode variable into a binary attribute: 1 if producer is present, 0 if not
df_echo.loc[df_echo.album.producer.isna(),('album','producer')] = 0
df_echo.loc[df_echo.album.producer.notna(),('album','producer')] = 1

In [ ]:
#drop entire columns
df_echo.drop(columns = [('album','information')], inplace = True)
df_echo.drop(columns = [('album','tags')], inplace = True)
df_echo.drop(columns= [('album','date_created')], inplace =  True)

df_echo.drop(columns = [('artist','active_year_begin')], inplace = True)
df_echo.drop(columns = [('artist','active_year_end')], inplace = True)
df_echo.drop(columns = [('artist','associated_labels')], inplace = True)
df_echo.drop(columns = [('artist','bio')], inplace = True)
df_echo.drop(columns = [('artist','date_created')], inplace = True)
df_echo.drop(columns = [('artist','location')], inplace = True)
df_echo.drop(columns = [('artist','latitude')], inplace = True)
df_echo.drop(columns = [('artist','longitude')], inplace = True)
df_echo.drop(columns = [('artist','name')], inplace = True)
df_echo.drop(columns = [('artist','website')], inplace = True)
df_echo.drop(columns = [('artist','wikipedia_page')], inplace = True)

df_echo.drop(columns = [('track','composer')], inplace = True)
df_echo.drop(columns = [('track','date_created')], inplace = True)
df_echo.drop(columns = [('track','lyricist')], inplace = True)
df_echo.drop(columns = [('track','tags')], inplace = True)
df_echo.drop(columns = [('track','information')], inplace = True)
df_echo.drop(columns = [('track','publisher')], inplace = True)
df_echo.drop(columns = [('track','number')], inplace = True)
df_echo.drop(columns = [('track','title')], inplace = True)

In [ ]:
#use track.date_recorded to impute album.date_released. Then drop track.date_recorded
df_echo.loc[(df_echo.track.date_recorded.notna()) & (df_echo.album.date_released.isna()), ('album','date_released')] = df_echo.loc[(df_echo.track.date_recorded.notna()) & (df_echo.album.date_released.isna()), ('track','date_recorded')]
df_echo.drop(columns = [('track','date_recorded')], inplace = True)

#bit_rate
df_echo.loc[:,('track','bit_rate')] = df_echo.track.bit_rate/1000
df_echo.loc[df_echo.track.bit_rate < 0, ('track', 'bit_rate')] = 320

#genres
#1 -> drop rows with no genres at all
df_echo.drop(df_echo.loc[df_echo.track.genres_all.str.len() == 0].index, inplace = True)
#2 -> clean genre top  all: abbiamo creato una nuova colonna uguale a genres_all dalla quale abbiamo rimosso tutti gli id che non sono top level and ADD NEW COLUMN CREATED TO df_echo
genres_top_list = list(set(genres_df.top_level.to_list()))
genre_top_all = pd.DataFrame(df_echo.track.genres_all)
for i, lista in genre_top_all.iterrows():
    temp_list = []
    for item in lista[0]:
        if item not in genres_top_list:
            temp_list.append(item)
    for j in temp_list:
        lista[0].remove(j)
df_echo.loc[:,'track'] = df_echo.track.join(genre_top_all.rename(columns={'genres_all' : 'genres_top_all'}), how = 'outer')
#3 -> fill genre_top empty with "Mixed" (because they have mixed genres)
df_echo.loc[:,('track','genre_top')].fillna('Mixed',inplace = True)

In [ ]:
#building echo_track dataframe
df_echo.columns = df_echo.columns.map('_'.join).str.strip('_')
echo_to_merge = echonest_df.iloc[:,:8].copy()
echo_to_merge.columns = echonest_df.iloc[:,:8].columns.droplevel(0).map('_'.join).str.strip('_')
echo_track = pd.merge(echo_to_merge, df_echo, on ='track_id')

In [ ]:
#LE LISTE CON VALORI NULLI SONO LISTE VUOTE
list_null = ['artist_members','artist_related_projects']

for column in echo_track.loc[:,list_null]:
    echo_track.loc[echo_track[column].notnull(), column] = echo_track.loc[echo_track[column].notnull(), column].apply(lambda x: x.split(", "))
    echo_track.loc[echo_track[column].isnull(), column] = echo_track.loc[echo_track[column].isnull(),column].apply(lambda x: [])

In [ ]:
#REPLEACING MISSING VALUES BY IDS
echo_track.loc[:,'track_license'] = echo_track.groupby('artist_id')['track_license'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
echo_track.loc[:,'album_type'] = echo_track.groupby('album_id')['album_type'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
echo_track.loc[:,'track_language_code'] = echo_track.groupby('album_id')['track_language_code'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
echo_track.loc[:,'track_language_code'] = echo_track.groupby('artist_id')['track_language_code'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
echo_track.loc[:,'album_date_released'] = echo_track.groupby('album_id')['album_date_released'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)

In [ ]:
echo_track.to_csv('echo_track.csv', index = True)